In [1]:
# 크롤링한 csv 파일 불러오기
import pandas as pd
df = pd.read_csv('변환파일.csv')
df.head()

,Column1,Title,Detail,new
0,0,"출산휴가, 육아휴직 계산 한번 확인부탁드려요~~",1. 임신중 육아휴직 4개월 : 2023. 7.12. ~ 11.13.\n2. 출산휴...,"출산휴가, 육아휴직 계산 한번 확인부탁드려요~~ 1. 임신중 육아휴직 4개월 : 2..."
1,1,3+3 육아휴직 잘 아시는분!,3+3 육아휴직 쓰려고하는데\n제가 지금 육휴를 쓰고있는 상황에\n아이가 6월10일...,3+3 육아휴직 잘 아시는분! 3+3 육아휴직 쓰려고하는데\n제가 지금 육휴를 쓰고...
2,2,육아휴직 중 연차 발생,안녕하세요!\n2021년 12월 1일~2022년 2월 28일 : 90일 출산휴가\n...,육아휴직 중 연차 발생 안녕하세요!\n2021년 12월 1일~2022년 2월 28일...
3,3,"둘째계획, 쭉 쉬었다가 복직? 복직했다가 나중에 육아휴직?",안녕하세요.\n첫째 키우고 있고 육휴 중이예요.\n곧 복직을 앞두고있는데 저는 둘째...,"둘째계획, 쭉 쉬었다가 복직? 복직했다가 나중에 육아휴직? 안녕하세요.\n첫째 키우..."
4,4,부부육아휴직 3개월 나누어쓰기 가능한가요??,부부육아휴직 3개월 나누어쓰기 가능한가요??\n남편 공동육아휴직으로 3개월쓸생각인데...,부부육아휴직 3개월 나누어쓰기 가능한가요?? 부부육아휴직 3개월 나누어쓰기 가능한가...


In [2]:
import re
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings(action='ignore')
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel

In [3]:
# 제목과 내용을 합친 'post' column
df = pd.DataFrame({'post': df['new']})
df.head()

,post
0,"출산휴가, 육아휴직 계산 한번 확인부탁드려요~~ 1. 임신중 육아휴직 4개월 : 2..."
1,3+3 육아휴직 잘 아시는분! 3+3 육아휴직 쓰려고하는데\n제가 지금 육휴를 쓰고...
2,육아휴직 중 연차 발생 안녕하세요!\n2021년 12월 1일~2022년 2월 28일...
3,"둘째계획, 쭉 쉬었다가 복직? 복직했다가 나중에 육아휴직? 안녕하세요.\n첫째 키우..."
4,부부육아휴직 3개월 나누어쓰기 가능한가요?? 부부육아휴직 3개월 나누어쓰기 가능한가...


In [4]:
# 텍스트 전처리
def remove_urls(text):
    " removes urls" # 함수의 목적과 사용법을 설명한느 docstring
    url_pattern = re.compile(r'https?://\S+|www\.\S+') # URL 패턴을 컴파일 정규 표현식은 https:// 또는 http:// 또는 www.으로 시작하는 문자열
    # https?://\S+: s?는 s문자가 0번 또는 1번 나타날 수 있음을 의미, \S+는 공백이 아닌 문자가 1번 이상 반복됨을 의미
    # www\.\S+: \.는 .문자 자체 의미
    return url_pattern.sub(r'', text) # text에서 URL 패턴과 일치하는 부분을 빈 문자열로 대체

def remove_emails(text):
    email_pattern = re.compile('\S*@\S*\s?') # \S*: 공백이 아닌 문자가 0개 이상, 그 뒤에 @, 다시 공백이 아닌 문자가 0개 이상 나오고(\S*)
    # \s?: 공백 문자가 0개 또는 1개 나오는 문자열을 찾음, 공백 문자: 스페이스바, 탭, 줄 바꿈 문자 등
    return email_pattern.sub(r'', text)

def remove_new_line(text):
    return re.sub('\s+', ' ', text) # 연속된 공백 문자를 찾아서 하나의 공백 문자로 바꿈.

def remove_non_alpha(text):
    return re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", str(text)) # [^A-Za-z]+ : 알파벳이 아닌 문자 찾아서 하나의 공백 문자로 바꿈. 

def preprocess_text(text):
    t = remove_urls(text)
    t = remove_emails(t)
    t = remove_new_line(t)
    t = remove_non_alpha(t)
    return t

def remove_stopwords(text, stopwords): # stopwords를 제거하는 함수.
    return " ".join([word for word in str(text).split() if word not in stopwords])
#     # 문자열을 단어 단위로 분리하여 stopwords가 아닌 단어만 선택하여 문자열로 다시 합침.

df['post_preprocessed'] = df['post'].apply(preprocess_text)
# # post 열에 있는 모든 값을 preprocess_text 함수에 적용 -> 전처리된 값들을 모두 소문자로 변환

df.head()

,post,post_preprocessed
0,"출산휴가, 육아휴직 계산 한번 확인부탁드려요~~ 1. 임신중 육아휴직 4개월 : 2...",출산휴가 육아휴직 계산 한번 확인부탁드려요 1 임신중 육아휴직 4개월 2023 7...
1,3+3 육아휴직 잘 아시는분! 3+3 육아휴직 쓰려고하는데\n제가 지금 육휴를 쓰고...,33 육아휴직 잘 아시는분 33 육아휴직 쓰려고하는데 제가 지금 육휴를 쓰고있는 상...
2,육아휴직 중 연차 발생 안녕하세요!\n2021년 12월 1일~2022년 2월 28일...,육아휴직 중 연차 발생 안녕하세요 2021년 12월 1일2022년 2월 28일 9...
3,"둘째계획, 쭉 쉬었다가 복직? 복직했다가 나중에 육아휴직? 안녕하세요.\n첫째 키우...",둘째계획 쭉 쉬었다가 복직 복직했다가 나중에 육아휴직 안녕하세요 첫째 키우고 있고 ...
4,부부육아휴직 3개월 나누어쓰기 가능한가요?? 부부육아휴직 3개월 나누어쓰기 가능한가...,부부육아휴직 3개월 나누어쓰기 가능한가요 부부육아휴직 3개월 나누어쓰기 가능한가요 ...


In [5]:
# 불용어 처리
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

stop_words = "우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 저것 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓"
stop_words=stop_words.split(' ')
stop_words
def remove_stopwords(text, stopwords): # stopwords를 제거하는 함수.
    return " ".join([word for word in str(text).split() if word not in stopwords])

df['post_final'] = df['post_preprocessed'].apply(lambda post: remove_stopwords(post, stop_words))

df

,post,post_preprocessed,post_final
0,"출산휴가, 육아휴직 계산 한번 확인부탁드려요~~ 1. 임신중 육아휴직 4개월 : 2...",출산휴가 육아휴직 계산 한번 확인부탁드려요 1 임신중 육아휴직 4개월 2023 7...,출산휴가 육아휴직 계산 한번 확인부탁드려요 1 임신중 육아휴직 4개월 2023 71...
1,3+3 육아휴직 잘 아시는분! 3+3 육아휴직 쓰려고하는데\n제가 지금 육휴를 쓰고...,33 육아휴직 잘 아시는분 33 육아휴직 쓰려고하는데 제가 지금 육휴를 쓰고있는 상...,33 육아휴직 아시는분 33 육아휴직 쓰려고하는데 제가 육휴를 쓰고있는 상황에 아이...
2,육아휴직 중 연차 발생 안녕하세요!\n2021년 12월 1일~2022년 2월 28일...,육아휴직 중 연차 발생 안녕하세요 2021년 12월 1일2022년 2월 28일 9...,육아휴직 연차 발생 안녕하세요 2021년 12월 1일2022년 2월 28일 90일 ...
3,"둘째계획, 쭉 쉬었다가 복직? 복직했다가 나중에 육아휴직? 안녕하세요.\n첫째 키우...",둘째계획 쭉 쉬었다가 복직 복직했다가 나중에 육아휴직 안녕하세요 첫째 키우고 있고 ...,둘째계획 쭉 쉬었다가 복직 복직했다가 나중에 육아휴직 안녕하세요 첫째 키우고 있고 ...
4,부부육아휴직 3개월 나누어쓰기 가능한가요?? 부부육아휴직 3개월 나누어쓰기 가능한가...,부부육아휴직 3개월 나누어쓰기 가능한가요 부부육아휴직 3개월 나누어쓰기 가능한가요 ...,부부육아휴직 3개월 나누어쓰기 가능한가요 부부육아휴직 3개월 나누어쓰기 가능한가요 ...
...,...,...,...
4120,육아휴직 급여 신청 문의 4/1 부터 육아휴직을 시작했어요\n회사에서 4/13 신청...,육아휴직 급여 신청 문의 41 부터 육아휴직을 시작했어요 회사에서 413 신청을 해...,육아휴직 급여 신청 문의 41 부터 육아휴직을 시작했어요 회사에서 413 신청을 해...
4121,저 복직할까요 육아휴직 연장할까요..ㅜㅜ 안녕하세요\n6월 복직 앞두고 이런저런 고...,저 복직할까요 육아휴직 연장할까요 안녕하세요 6월 복직 앞두고 이런저런 고민이 너무...,저 복직할까요 육아휴직 연장할까요 안녕하세요 6월 복직 앞두고 이런저런 고민이 너무...
4122,남편 육아휴직 후 생계 잘 이어가시나요? 저는 현재 주부고 아이셋있어요\n남편이 많...,남편 육아휴직 후 생계 잘 이어가시나요 저는 현재 주부고 아이셋있어요 남편이 많이 ...,남편 육아휴직 생계 이어가시나요 저는 현재 주부고 아이셋있어요 남편이 많이 쉬고 싶...
4123,육아휴직 3+3 사용하신분 궁금해요 남편이 먼저 육아휴직 2월달에 시작했구요\n저는...,육아휴직 33 사용하신분 궁금해요 남편이 먼저 육아휴직 2월달에 시작했구요 저는 3...,육아휴직 33 사용하신분 궁금해요 남편이 먼저 육아휴직 2월달에 시작했구요 저는 3...


In [6]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [7]:
from konlpy.tag import Hannanum
 
hannanum = Hannanum()

In [8]:
# 명사 단위로 쪼개기
df['nouns'] = df['post_final'].apply(lambda post: hannanum.nouns(post))
df['nouns'] = df['nouns'].apply(lambda post: " ".join(post))
df.head()

,post,post_preprocessed,post_final,nouns
0,"출산휴가, 육아휴직 계산 한번 확인부탁드려요~~ 1. 임신중 육아휴직 4개월 : 2...",출산휴가 육아휴직 계산 한번 확인부탁드려요 1 임신중 육아휴직 4개월 2023 7...,출산휴가 육아휴직 계산 한번 확인부탁드려요 1 임신중 육아휴직 4개월 2023 71...,출산휴가 육아휴직 계산 한번 확인부탁드 1 임신중 육아휴직 4개월 2023 712 ...
1,3+3 육아휴직 잘 아시는분! 3+3 육아휴직 쓰려고하는데\n제가 지금 육휴를 쓰고...,33 육아휴직 잘 아시는분 33 육아휴직 쓰려고하는데 제가 지금 육휴를 쓰고있는 상...,33 육아휴직 아시는분 33 육아휴직 쓰려고하는데 제가 육휴를 쓰고있는 상황에 아이...,33 육아휴직 분 33 육아휴직 저 육휴 상황 아이 6월10일생 남편 610 전 육...
2,육아휴직 중 연차 발생 안녕하세요!\n2021년 12월 1일~2022년 2월 28일...,육아휴직 중 연차 발생 안녕하세요 2021년 12월 1일2022년 2월 28일 9...,육아휴직 연차 발생 안녕하세요 2021년 12월 1일2022년 2월 28일 90일 ...,육아휴직 연차 발생 안녕 2021년 12월 1일2022년 2월 28일 90일 출산휴...
3,"둘째계획, 쭉 쉬었다가 복직? 복직했다가 나중에 육아휴직? 안녕하세요.\n첫째 키우...",둘째계획 쭉 쉬었다가 복직 복직했다가 나중에 육아휴직 안녕하세요 첫째 키우고 있고 ...,둘째계획 쭉 쉬었다가 복직 복직했다가 나중에 육아휴직 안녕하세요 첫째 키우고 있고 ...,둘째계획 복직 복직 나중 육아휴직 안녕 첫째 육휴 중 복직 저 둘째 계획 생각 막막...
4,부부육아휴직 3개월 나누어쓰기 가능한가요?? 부부육아휴직 3개월 나누어쓰기 가능한가...,부부육아휴직 3개월 나누어쓰기 가능한가요 부부육아휴직 3개월 나누어쓰기 가능한가요 ...,부부육아휴직 3개월 나누어쓰기 가능한가요 부부육아휴직 3개월 나누어쓰기 가능한가요 ...,부부육아휴직 3개월 나누어쓰 가능 부부육아휴직 3개월 나누어쓰 가능 남편 공동육아휴...


In [9]:
# konlpy 돌린 파일 저장
df.to_csv('konlpy이후.csv', encoding="utf-8")
print(df)

                                                   post   
0     출산휴가, 육아휴직 계산 한번 확인부탁드려요~~ 1. 임신중 육아휴직 4개월 : 2...  \
1     3+3 육아휴직 잘 아시는분! 3+3 육아휴직 쓰려고하는데\n제가 지금 육휴를 쓰고...   
2     육아휴직 중 연차 발생 안녕하세요!\n2021년 12월 1일~2022년 2월 28일...   
3     둘째계획, 쭉 쉬었다가 복직? 복직했다가 나중에 육아휴직? 안녕하세요.\n첫째 키우...   
4     부부육아휴직 3개월 나누어쓰기 가능한가요?? 부부육아휴직 3개월 나누어쓰기 가능한가...   
...                                                 ...   
4120  육아휴직 급여 신청 문의 4/1 부터 육아휴직을 시작했어요\n회사에서 4/13 신청...   
4121  저 복직할까요 육아휴직 연장할까요..ㅜㅜ 안녕하세요\n6월 복직 앞두고 이런저런 고...   
4122  남편 육아휴직 후 생계 잘 이어가시나요? 저는 현재 주부고 아이셋있어요\n남편이 많...   
4123  육아휴직 3+3 사용하신분 궁금해요 남편이 먼저 육아휴직 2월달에 시작했구요\n저는...   
4124  육아휴직 급여 신청후 얼마나걸리나요? 10일 전 신청했고 처리중으로 뜨고요 어제 막...   

                                      post_preprocessed   
0     출산휴가 육아휴직 계산 한번 확인부탁드려요 1 임신중 육아휴직 4개월  2023 7...  \
1     33 육아휴직 잘 아시는분 33 육아휴직 쓰려고하는데 제가 지금 육휴를 쓰고있는 상...   
2     육아휴직 중 연차 발생 안녕하세요 2021년 12월 1일2022년 2월 28일  9...   
3     둘째계획 쭉 쉬었다가 복직 복직했다가 나중에 육아휴직 안녕하세요 첫째 키우고 있고 ...